In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
inputs = keras.Input(shape=(28,28))
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')
dense2 = keras.layers.Dense(10, activation='softmax', name='category_output')
dense3 = keras.layers.Dense(1, activation='sigmoid', name='left_right_output')


In [3]:
x = flatten(inputs)
x = dense1(x)
output1 = dense2(x)
output2 = dense3(x)
model = keras.Model(inputs = inputs, outputs = [output1, output2], name = 'mnist_model')

In [4]:
model.summary()

Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 784)       │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    100,480 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_output     │ (None, 10)        │      1,290 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ left_right_output   │ (None, 1)         │        129 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 101,899 (398.04 KB)

 Trainable params: 101,899 (398.04 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss2 = keras.losses.BinaryFocalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)  # ✅ use learning_rate
metrics={
    "category_output": "accuracy",
    "left_right_output": "accuracy"
}
losses = {
    "category_output": loss1,
    "left_right_output": loss2,
}
model.compile(loss=losses, optimizer=optim, metrics=metrics)

In [6]:
mnist = keras.datasets.mnist
(x_train,y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

#left = 0, Right = 1
y_left_right = np.zeros(y_train.shape, dtype = np.uint8)
for idx,y in enumerate(y_train):
    if y > 5:
        y_left_right[idx] = 1
print(y_train.dtype, y_train[0:20])
print(y_left_right.dtype, y_left_right[0:20])

y = {"category_output": y_train,
     "left_right_output": y_left_right}

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
uint8 [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]
uint8 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]


In [7]:
model.fit(x_train,y=y, epochs = 5, batch_size=64, verbose=2)

Epoch 1/5
938/938 - 6s - 6ms/step - category_output_accuracy: 0.9117 - category_output_loss: 0.3125 - left_right_output_accuracy: 0.9248 - left_right_output_loss: 0.0522 - loss: 0.3647
Epoch 2/5
938/938 - 3s - 3ms/step - category_output_accuracy: 0.9593 - category_output_loss: 0.1408 - left_right_output_accuracy: 0.9644 - left_right_output_loss: 0.0281 - loss: 0.1690
Epoch 3/5
938/938 - 2s - 2ms/step - category_output_accuracy: 0.9708 - category_output_loss: 0.0993 - left_right_output_accuracy: 0.9699 - left_right_output_loss: 0.0233 - loss: 0.1226
Epoch 4/5
938/938 - 2s - 2ms/step - category_output_accuracy: 0.9779 - category_output_loss: 0.0756 - left_right_output_accuracy: 0.9740 - left_right_output_loss: 0.0202 - loss: 0.0956
Epoch 5/5
938/938 - 3s - 3ms/step - category_output_accuracy: 0.9821 - category_output_loss: 0.0603 - left_right_output_accuracy: 0.9773 - left_right_output_loss: 0.0174 - loss: 0.0776


In [8]:
prediction = model.predict(x_test)
len(prediction)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2

In [9]:
prediction_category = prediction[0]
prediction_left_right = prediction[1]

pr_cat = prediction_category[0:20]
prediction_left_right = prediction_left_right[0:20]
labels_cat = np.argmax(pr_cat, axis=1)
labels_left_right = np.array([1 if p >= 0.5 else 0 for p in prediction_left_right])
print(labels_cat)
print(labels_left_right)

[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
[1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0]
